In [1]:
import re
import string
import math
from nltk.probability import (ConditionalFreqDist, ConditionalProbDist, LidstoneProbDist)
from nltk import pos_tag
# from classifiers.DecisionListClassifier import DecisionListClassifier
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.metrics import ConfusionMatrix
from collections import Counter
import re
import string
import random

In [2]:
class DecisionListClassifier(object):
   

    def __init__(self, target_word, sense_pairs, max_collocation=10):
   
        word_senses = list(set([w[0] for w in sense_pairs]))

        word_sense_count = 0
        word_sense_star_count = 0
        word_sense_majority = ''
        for w in sense_pairs:
            if w == word_senses[0]:
                word_sense_count +=1
            else:
                word_sense_star_count +=1

        if word_sense_count/(word_sense_star_count+word_sense_count) > \
            word_sense_star_count/(word_sense_star_count+word_sense_count) :
            word_sense_majority = word_senses[0]
        else:
            word_sense_majority = word_senses[1]
        self.word = target_word
        self.word_sense = word_senses[0]
        self.word_sense_star = word_senses[1]
        self.majority_word_sense = word_sense_majority
        dl = self.train(target_word, sense_pairs, max_collocation)
        self._decision_list = dl
        self.evaluation_result = {}

    def __repr__(self):
        return '<DecisionListClassifier: %s rules>' % (len(self._decision_list))

    def __getitem__(self, i):
        return self._decision_list[i]

    def evaluate(self, target_word, tokens, max_collocation=10):
        features = []
        features += self.generate_collocations(target_word, tokens, max_collocation)

        pos = self.parts_of_speech(target_word, tokens)
        for g in self.generate_collocations(target_word, pos, max_collocation):
            if g != target_word: 
                features += [g]

        max_log = 0.0
        sense = self.majority_word_sense
        for f in features:
            for rule in self._decision_list:
                if rule[1] == f:
                    if rule[2] > max_log:
                        max_log = rule[2]
                        sense = rule[0]

        return sense

    def train(self, target_word, sense_pairs, max_collocation):
        freqs = ConditionalFreqDist()

        for pair in sense_pairs:
            for g in self.generate_collocations(target_word, pair[1], max_collocation):
                freqs[g][pair[0]] += 1
            pos = self.parts_of_speech(target_word, pair[1])
            for g in self.generate_collocations(target_word, pos, max_collocation):
                if g != self.word:
                    freqs[g][pair[0]] += 1

        probs = ConditionalProbDist(freqs, LidstoneProbDist, 0.2)

        dl = []
        for feature in probs.conditions():
            self.add_to_decision_list(dl, probs, feature)
        dl.sort(key=lambda r: r[2], reverse=True)

        return dl

    def add_to_decision_list(self, dl, probs, feature):
        prob = probs[feature].prob(self.word_sense) # Pr(sense1|feature)
        prob_star = probs[feature].prob(self.word_sense_star) # Pr(sense2|feature)
        d = math.log(prob/prob_star)
        if d == 0:
            dl.append((self.majority_word_sense, feature, 0))
        else:
            # can tell what sense feature is for be +/- value
            sense = self.word if d > 0 else self.word_sense_star
            dl.append((sense, feature, abs(d)))

    def parts_of_speech(self, target_word, tokens):
        b = tokens.index(target_word)
        pos = [w[1] for w in pos_tag(tokens)]
        pos[b] = target_word
        return pos

    
    def generate_collocations(self, word, tokens, max_c):
        def ngrams_h(c, l, r, acc):
            if c == 0:
                return acc
            acc.append('_'.join(tokens[l:r]))
            return ngrams_h(c-1, l+1, r+1, acc)

        w = tokens.index(word) #ValueError
        ngrams = []
        for c in range(max_c, 0, -1):
            ngrams = ngrams_h(c+1, w-c, w+1, [])

        ngrams += [word]
        return ngrams

In [3]:
def preprocess_data(data):
    c = []
    stop_set = set(stopwords.words('english'))
    data = [l for l in data.split("\n") if len(l)]
    senses = []

    for line in data:

        line = line.strip()
        s = line.index(":")
        line = (line[:s], line[s:])

        line = (line[0], word_tokenize(line[1]))
        line = (line[0], [w for w in line[1] if w not in stop_set])
        line = (line[0], [re.sub('['+string.punctuation+']', '', w) for w in line[1]])
        line = (line[0], [w for w in line[1] if len(w) > 2])
        line = (line[0], [w.lower() for w in line[1]])

        senses.append(line[0])
        c.append(line)

    return c

In [4]:
def evaluate(word, dlc, test_data, sample_size=3):

    result = {}

    word_senses = list(set([w[0] for w in test_data]))
    word_sense, word_sense_star = word_senses[0], word_senses[1]
    word_sense_count = 0
    word_sense_star_count = 0
    word_sense_majority = ''
    for w in test_data:
        if w[0] == word_sense:
            word_sense_count +=1
        else:
            word_sense_star_count +=1

    if word_sense_count/(word_sense_star_count+word_sense_count) > \
        word_sense_star_count/(word_sense_star_count+word_sense_count) :
        word_sense_majority = word_senses[0]
    else:
        word_sense_majority = word_senses[1]
    baseline_correct = 0
    result["majority_baseline"] = word_sense_majority
    for row in test_data:
        if row[0] == word_sense_majority:
            baseline_correct += 1
    result["majority_baseline_percent_correct"] = round(baseline_correct/len(test_data)*100, 2)
    result["correct_count"], result["incorrect_count"] = 0, 0
    guesses = [] # what we guessed
    actual = [] # what acutally was
    correctly_guessed = []
    incorrectly_guessed = []
    for row in test_data:
        g = dlc.evaluate(word, row[1])
        guesses.append(g)
        actual.append(row[0])
        if g == row[0]:
            result["correct_count"] += 1
            correctly_guessed.append(row[1])
        else:
            result["incorrect_count"] += 1
            incorrectly_guessed.append(row[1])

    result["correct_guess_sample"] = random.sample(correctly_guessed, sample_size)
    result["incorrect_guess_sample"] = random.sample(incorrectly_guessed, sample_size)
    result["percent_correct"] = round((result["correct_count"]/len(test_data))*100, 2)

    result["confusion_matrix"] = ConfusionMatrix(actual, guesses)

    true_pos = Counter()
    false_neg = Counter()
    false_pos = Counter()
    for i in [word_sense, word_sense_star]:
        for j in [word_sense, word_sense_star]:
            if i == j:
                true_pos[i] += result["confusion_matrix"][i,j]
            else:
                false_neg[i] += result["confusion_matrix"][i,j]
                false_pos[j] += result["confusion_matrix"][i,j]

    # precision
    result["precision_word"] = 0
    if float(true_pos[word_sense_star]+false_pos[word_sense_star]) != 0:
        result["precision_word_star"] = true_pos[word_sense_star] / float(true_pos[word_sense_star]+false_pos[word_sense_star])

    result["recall_word"] = true_pos[word_sense] / float(true_pos[word_sense]+false_neg[word_sense])
    result["recall_word_star"] = true_pos[word_sense_star] / float(true_pos[word_sense_star]+false_neg[word_sense_star])

    result["macro_precision"] = (float(result["recall_word"]) + float(result["recall_word_star"])) / 2.0
    result["macro_recall"] = (float(result["recall_word"]) + float(result["recall_word_star"])) / 2.0
    
    result["word_sense"] = word_sense
    result["word_sense_star"] = word_sense_star

    for i in result:
            print(i, ":  '", result[i], "'")

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/garga4/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
sense = 'bass'
train = preprocess_data(open('./bass.trn', 'r').read())
test = preprocess_data(open('./bass.tst', 'r').read())
dlc = DecisionListClassifier(sense, train)
result = evaluate(sense, dlc, test)
# report(dlc, result)

majority_baseline :  ' bass '
majority_baseline_percent_correct :  ' 56.0 '
correct_count :  ' 90 '
incorrect_count :  ' 10 '
correct_guess_sample :  ' [['timbres', 'single', 'muddy', 'bass', 'line', 'the', 'treble', 'fared'], ['rocks', 'wreck', 'bass', 'fishing', 'gear', 'saturday'], ['city', 'street', 'for', 'bass', 'drummers', 'bass', 'horn', 'players']] '
incorrect_guess_sample :  ' [['knobs', '303', 'bass', 'machine', 'could', 'squelch'], ['old', 'salt', 'knows', 'bass', 'frying', 'instead'], ['galvany', 'tenor', 'edward', 'perretti', 'bass', 'valentin', 'peitchinoff', 'less']] '
percent_correct :  ' 90.0 '
confusion_matrix :  '       |  *    |
      |  b  b |
      |  a  a |
      |  s  s |
      |  s  s |
------+-------+
*bass |<39> 5 |
 bass |  5<51>|
------+-------+
(row = reference; col = test)
 '
precision_word :  ' 0 '
precision_word_star :  ' 0.8863636363636364 '
recall_word :  ' 0.9107142857142857 '
recall_word_star :  ' 0.8863636363636364 '
macro_precision :  ' 0.8985389

In [7]:
sense = 'sake'
train = preprocess_data(open('./sake.trn', 'r').read())
test = preprocess_data(open('./sake.tst', 'r').read())
dlc = DecisionListClassifier(sense, train)
result = evaluate(sense, dlc, test)
# report(dlc, result)

majority_baseline :  ' sake '
majority_baseline_percent_correct :  ' 94.0 '
correct_count :  ' 94 '
incorrect_count :  ' 6 '
correct_guess_sample :  ' [['damage', 'investor', 'confidence', 'for', 'sake', 'bank', 'indonesia', 'credibility', 'sabirin'], ['hope', 'really', 'just', 'sake', 'sometime', '1940'], ['tuesday', 'night', 'for', 'sake', 'children']] '
incorrect_guess_sample :  ' [['growing', 'especially', 'premium', 'sake', 'category', 'sales', 'rose'], ['hallmarks', 'sake', 'industry', '20th', 'century'], ['type', 'fine', 'traditional', 'sake', 'rice']] '
percent_correct :  ' 94.0 '
confusion_matrix :  '       |  *    |
      |  s  s |
      |  a  a |
      |  k  k |
      |  e  e |
------+-------+
*sake | <.> 6 |
 sake |  .<94>|
------+-------+
(row = reference; col = test)
 '
precision_word :  ' 0 '
precision_word_star :  ' 0.94 '
recall_word :  ' 0.0 '
recall_word_star :  ' 1.0 '
macro_precision :  ' 0.5 '
macro_recall :  ' 0.5 '
word_sense :  ' *sake '
word_sense_star :  ' sa